In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

import pickle

In [10]:
# Load data

# data = pd.read_csv('SQLiV3.csv', encoding='utf-16')
data = pd.read_csv('SQLiV3.csv')

In [11]:
# Loại bỏ các cột không cần thiết
data_cleaned = data.drop(columns=['Unnamed: 2', 'Unnamed: 3'])

# Kiểm tra và xử lý các giá trị thiếu
data_cleaned.dropna(subset=['Sentence', 'Label'], inplace=True)

# Chuyển văn bản về dạng chữ thường và loại bỏ khoảng trắng dư thừa
data_cleaned['Sentence'] = data_cleaned['Sentence'].str.strip().str.lower()

# Kiểm tra dữ liệu sau khi làm sạch
data_cleaned.head()


,Sentence,Label
0,""" or pg_sleep ( __time__ ) --",1
2,and 1 = utl_inaddr.get_host_address ( (...,1
3,select * from users where id = '1' or @ @1 ...,1
4,"select * from users where id = 1 or 1#"" ( ...",1
5,select name from syscolumns where id = (...,1


In [18]:
# Loại bỏ các dòng có giá trị NaN trong cả cột 'Sentence' và 'Label'
data_cleaned = data.dropna(subset=['Sentence', 'Label'])

# Tiến hành chia dữ liệu lại
X_train, X_test, y_train, y_test = train_test_split(data_cleaned['Sentence'], data_cleaned['Label'], test_size=0.2, random_state=42)

# Tiến hành vector hóa
vectorizer = TfidfVectorizer(max_features=1000)

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train the model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_tfidf, y_train)


RandomForestClassifier(random_state=42)

In [19]:
# Train the model 
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_tfidf, y_train)

RandomForestClassifier(random_state=42)

In [22]:
from sklearn.metrics import accuracy_score, classification_report

# Dự đoán trên tập kiểm tra
y_pred = model.predict(X_test_tfidf)

# Đánh giá mô hình
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9921696574225123
Classification Report:
                                                           precision    recall  f1-score   support

                                 EXEC SelectAllCustomers       0.00      0.00      0.00         1
                                                      --       1.00      1.00      1.00         1
            CREATE OR REPLACE VIEW [Brazil Customers] AS       0.00      0.00      0.00         0
                     CREATE OR REPLACE VIEW view_name AS       0.00      0.00      0.00         1
           CREATE VIEW [Products Above Average Price] AS       0.00      0.00      0.00         1
                                 EXEC SelectAllCustomers       0.00      0.00      0.00         1
 SELECT * FROM CustomersFETCH FIRST 50 PERCENT ROWS ONLY       0.00      0.00      0.00         1
                                                       0       0.99      1.00      0.99      3852
                                                       1       1

c:\Users\21522\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\21522\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\21522\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 

In [23]:
# Lưu lại model
with open('sql.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

In [24]:
# Lưu lại file vector
with open('tfidf_vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)

In [27]:
def sql_detect(query):
    with open('sql.pkl', 'rb') as model_file:
        load_model=pickle.load(model_file)
    
    with open('tfidf_vectorizer.pkl', 'rb') as vectorize_file:
        load_vectorizer = pickle.load(vectorize_file)
    
    query_tfidf=load_vectorizer.transform([query])

    predict = load_model.predict(query_tfidf)

    return "SQL Injection Detected" if predict[0]==1 else "Safe"

print(sql_detect(" select * from users where id  =  '1' or @ @1  =  1 union select 1,version  (    )   -- 1'"))

Safe
